Run `apt-get update && apt-get install ffmpeg libsm6 libxext6  -y` in Terminal

In [ ]:
!nvidia-smi

In [1]:
!pip install ikomia numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.3/148.3 MB 9.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 61.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 64.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 78.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 13.8 MB/s

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home


In [3]:
%cd {HOME}

DATASET_DIR = f'{HOME}/dataset'

!curl -L "https://storage.kuckuck.cam/datasets/v1/train.zip" > train.zip; unzip -q train.zip -d "{DATASET_DIR}"; rm train.zip
!curl -L "https://storage.kuckuck.cam/datasets/v1/val.zip" > val.zip; unzip -q val.zip -d "{DATASET_DIR}"; rm val.zip
!curl -L "https://storage.kuckuck.cam/datasets/v1/test.zip" > test.zip; unzip -q test.zip -d "{DATASET_DIR}"; rm test.zip

!mkdir all
!cp -r "{DATASET_DIR}"/test/*/* $(pwd)/all/
!cp -r "{DATASET_DIR}"/val/*/* $(pwd)/all/
!cp -r "{DATASET_DIR}"/train/*/* $(pwd)/all/

!echo "Files in /all directory:"
!ls $(pwd)/all/ | wc -l

!cp -r $(pwd)/dataset/train/classes.txt $(pwd)/classes.txt

/home
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1845M  100 1845M    0     0  19.0M      0  0:01:36  0:01:36 --:--:-- 19.4M:00:59  0:00:39 19.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  279M  100  279M    0     0  17.9M      0  0:00:15  0:00:15 --:--:-- 20.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  516M  100  516M    0     0  16.6M      0  0:00:31  0:00:31 --:--:-- 19.3M
cp: will not overwrite just-created '/home/all/Icon' with '/home/dataset/val/labels/Icon'
cp: will not overwrite just-created '/home/all/Icon' with '/home/dataset/train/labels/Icon'
Files in /all directory:
17081


In [5]:
%load_ext tensorboard
%tensorboard --logdir /root/Ikomia/Tensorboard

In [6]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils import ik
import os

#----------------------------- Step 1 -----------------------------------#
# Create a workflow which will take your dataset as input and
# train a YOLOv7 model on it
#------------------------------------------------------------------------#
wf = Workflow()

#----------------------------- Step 2 -----------------------------------#
# First you need to convert darknet YOLO format to IKOMIA format.
# Add an Ikomia dataset converter to your workflow.
#------------------------------------------------------------------------#
dataset_yolo = wf.add_task(ik.dataset_yolo(dataset_folder=os.getcwd()+"/all", class_file=os.getcwd()+"/classes.txt"))

#----------------------------- Step 3 -----------------------------------#
# Then, you want to train a YOLOv7 model.
# Add YOLOv7 training algorithm to your workflow
#------------------------------------------------------------------------#
yolo7 = wf.add_task(ik.train_yolo_v7(output_folder=os.getcwd()+"/ikomia", batch_size="12", epochs="100"), auto_connect=True)# <-- Reduce the batch size if you encounter some "Cuda out of memory"

#----------------------------- Step 4 -----------------------------------#
# Execute your workflow.
# It automatically runs all your tasks sequentially.
#------------------------------------------------------------------------#
wf.run()

ModuleNotFoundError: No module named 'ikomia.lib.pycore'

# Test

In [5]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils import ik

# Create your workflow for YOLO inference
wf = Workflow()

# Add YOLO v7 to your workflow
yolov7 = wf.add_task(ik.infer_yolo_v7(model_weight_file=os.getcwd()+"/folder_name/weights/best.pt", conf_thres="0.25"), auto_connect=True) # <-- replace "folder_name" by the correct folder name (/content/ for Google Colab)

ModuleNotFoundError: No module named 'ikomia.lib.pycore'

In [ ]:
from ikomia.utils.displayIO import display
from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Apply YOLO v7 on your image
# By default, YOLOv7 runs with a pre-trained model based on COCO
# To use your custom model, set the parameters in the previous cell
wf.run_on(path=os.getcwd()+"/train/yt1s-com---Resume-Tottenham-20-West-Bromwich-Premier-League-J23_720p-fps-2_00044_jpeg_jpg.rf.9de8afe0132987f478d16d536e768a67.jpg")

# Get YOLO v7 image result
img_bbox = yolov7.get_image_with_graphics()

# Display in Colab
display(img_bbox)